In [9]:
import psycopg2
from psycopg2.extras import RealDictCursor
import pandas as pd
import uuid

In [25]:
conn = psycopg2.connect(
    host='ud-adp-rolling.cbyis8su6n0i.us-east-2.rds.amazonaws.com',
    user='postgres',
    password='liammurphybbm2',
    port=5432
)

cursor = conn.cursor(cursor_factory=RealDictCursor)

# BBM4 (2023)

### Raw Data -> ADP Data

In [2]:
# very large file, takes a long time to load
bbm4_data = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\large data\best_ball_mania_iv_2023_r1_results_pick_by_pick.csv', delimiter=',')

In [3]:
print(bbm4_data.columns)
bbm4_data.head(5)

Index(['draft_id', 'user_id', 'username', 'draft_created_time',
       'draft_filled_time', 'draft_time', 'draft_completed_time', 'clock',
       'draft_entry_id', 'tournament_entry_id',
       'tournament_round_draft_entry_id', 'tournament_round_number',
       'player_name', 'player_id', 'position_name', 'projection_adp', 'source',
       'pick_order', 'overall_pick_number', 'team_pick_number',
       'pick_created_time', 'pick_points', 'roster_points', 'made_playoffs'],
      dtype='object')


,draft_id,user_id,username,draft_created_time,draft_filled_time,draft_time,draft_completed_time,clock,draft_entry_id,tournament_entry_id,...,position_name,projection_adp,source,pick_order,overall_pick_number,team_pick_number,pick_created_time,pick_points,roster_points,made_playoffs
0,65219f4a-267e-4035-806a-646a3c1a846e,1a32930d-dbc3-412b-b4d5-8776feb34790,nohalfsteppin,2023-04-25T03:41:37Z,2023-09-03T18:54:09Z,2023-09-03T18:55:15Z,2023-09-03T19:42:23Z,30,a07d4470-1381-4151-b439-c13b67fc9e1c,375e1161-0892-4b8d-a00f-3a5936f4b23c,...,WR,8.57,user,6,6,1,2023-09-03T18:56:23Z,171.10,2091.34,0
1,65219f4a-267e-4035-806a-646a3c1a846e,1a32930d-dbc3-412b-b4d5-8776feb34790,nohalfsteppin,2023-04-25T03:41:37Z,2023-09-03T18:54:09Z,2023-09-03T18:55:15Z,2023-09-03T19:42:23Z,30,a07d4470-1381-4151-b439-c13b67fc9e1c,375e1161-0892-4b8d-a00f-3a5936f4b23c,...,QB,27.26,user,6,19,2,2023-09-03T18:58:43Z,247.84,2091.34,0
2,65219f4a-267e-4035-806a-646a3c1a846e,1a32930d-dbc3-412b-b4d5-8776feb34790,nohalfsteppin,2023-04-25T03:41:37Z,2023-09-03T18:54:09Z,2023-09-03T18:55:15Z,2023-09-03T19:42:23Z,30,a07d4470-1381-4151-b439-c13b67fc9e1c,375e1161-0892-4b8d-a00f-3a5936f4b23c,...,WR,34.95,user,6,30,3,2023-09-03T19:00:59Z,204.86,2091.34,0
3,65219f4a-267e-4035-806a-646a3c1a846e,1a32930d-dbc3-412b-b4d5-8776feb34790,nohalfsteppin,2023-04-25T03:41:37Z,2023-09-03T18:54:09Z,2023-09-03T18:55:15Z,2023-09-03T19:42:23Z,30,a07d4470-1381-4151-b439-c13b67fc9e1c,375e1161-0892-4b8d-a00f-3a5936f4b23c,...,WR,39.76,user,6,43,4,2023-09-03T19:02:58Z,52.10,2091.34,0
4,65219f4a-267e-4035-806a-646a3c1a846e,1a32930d-dbc3-412b-b4d5-8776feb34790,nohalfsteppin,2023-04-25T03:41:37Z,2023-09-03T18:54:09Z,2023-09-03T18:55:15Z,2023-09-03T19:42:23Z,30,a07d4470-1381-4151-b439-c13b67fc9e1c,375e1161-0892-4b8d-a00f-3a5936f4b23c,...,TE,53.16,user,6,54,5,2023-09-03T19:05:16Z,146.40,2091.34,0


In [37]:
bbm4_cond = bbm4_data.copy()[['player_name', 'player_id', 'position_name', 'projection_adp', 'pick_created_time']]
bbm4_cond['pick_created_time'] = pd.to_datetime(bbm4_cond['pick_created_time']) + pd.Timedelta(hours=-15)
bbm4_cond['pick_created_time'] = bbm4_cond['pick_created_time'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
bbm4_cond.head(5)

,player_name,player_id,position_name,projection_adp,pick_created_time
0,Stefon Diggs,25ea41e7-0529-48c6-ba65-7648bb7f9ed2,WR,8.57,2023-09-03T03:56:23Z
1,Josh Allen,99398db6-2ffb-42e9-bdb0-4312a399ab08,QB,27.26,2023-09-03T03:58:43Z
2,Keenan Allen,d74acd73-504a-4c9b-8aa6-fe602de560a8,WR,34.95,2023-09-03T04:00:59Z
3,Christian Watson,c108c752-92fd-491f-94ed-60b0b3cea7d8,WR,39.76,2023-09-03T04:02:58Z
4,T.J. Hockenson,660a9e72-4fc7-442d-ba62-257dbab39568,TE,53.16,2023-09-03T04:05:16Z


In [38]:
bbm4_cond_date = bbm4_cond.copy()
bbm4_cond_date['pick_created_time'] = bbm4_cond_date['pick_created_time'].str[:10]
bbm4_cond_date.head(5)

,player_name,player_id,position_name,projection_adp,pick_created_time
0,Stefon Diggs,25ea41e7-0529-48c6-ba65-7648bb7f9ed2,WR,8.57,2023-09-03
1,Josh Allen,99398db6-2ffb-42e9-bdb0-4312a399ab08,QB,27.26,2023-09-03
2,Keenan Allen,d74acd73-504a-4c9b-8aa6-fe602de560a8,WR,34.95,2023-09-03
3,Christian Watson,c108c752-92fd-491f-94ed-60b0b3cea7d8,WR,39.76,2023-09-03
4,T.J. Hockenson,660a9e72-4fc7-442d-ba62-257dbab39568,TE,53.16,2023-09-03


In [39]:
bbm4_05_10 = bbm4_cond_date[bbm4_cond_date['pick_created_time'] == '2023-08-14']
for player_name in bbm4_05_10['player_name'].unique():
    player_data = bbm4_05_10[bbm4_05_10['player_name'] == player_name]
    adp_counts = player_data['projection_adp'].value_counts()
    print(f"\n{player_name}:")
    for adp, count in adp_counts.items():
        print(f"  ADP {adp}: {count} times")


T.J. Hockenson:
  ADP 52.27: 657 times
  ADP 52.42: 2 times

Marquise Brown:
  ADP 61.87: 674 times
  ADP 61.53: 4 times

Sam LaPorta:
  ADP 150.11: 653 times
  ADP 150.0: 2 times

Raheem Mostert:
  ADP 158.07: 682 times
  ADP 158.67: 3 times

Tyreek Hill:
  ADP 4.13: 666 times
  ADP 4.1: 2 times

Derrick Henry:
  ADP 21.03: 669 times

Josh Allen:
  ADP 27.49: 667 times
  ADP 27.44: 3 times

Mike Williams:
  ADP 44.61: 643 times
  ADP 44.51: 2 times

Kenneth Walker III:
  ADP 53.6: 664 times
  ADP 53.59: 3 times

Michael Pittman Jr.:
  ADP 71.03: 654 times
  ADP 70.15: 2 times

George Kittle:
  ADP 62.62: 674 times
  ADP 62.64: 2 times

Elijah Moore:
  ADP 87.49: 659 times
  ADP 87.3: 2 times

AJ Dillon:
  ADP 101.18: 659 times
  ADP 101.38: 2 times

David Njoku:
  ADP 106.7: 652 times
  ADP 106.56: 2 times

Jakobi Meyers:
  ADP 126.91: 639 times
  ADP 126.73: 2 times

Adam Thielen:
  ADP 148.7: 635 times
  ADP 148.85: 3 times

Marvin Mims Jr.:
  ADP 145.02: 655 times
  ADP 144.48: 5 

In [42]:
dates = sorted(bbm4_cond_date['pick_created_time'].unique())
player_ids = bbm4_cond_date['player_id'].unique()

col_names = ['player_id', 'player_name', 'position'] + dates
bbm4_adp = pd.DataFrame(columns=col_names)

player_info = bbm4_cond_date.groupby('player_id').first()[['player_name', 'position_name']].reset_index()

for player_id in player_ids:
    player_data = player_info[player_info['player_id'] == player_id].iloc[0]
    row_data = [player_id, player_data['player_name'], player_data['position_name']] + [None] * len(dates)
    bbm4_adp.loc[len(bbm4_adp)] = row_data

In [46]:
for date in dates:
    bbm4_date = bbm4_cond_date[bbm4_cond_date['pick_created_time'] == date]
    for idx, row in bbm4_adp.iterrows():
        player_data = bbm4_date[bbm4_date['player_id'] == row['player_id']]
        adp_counts = player_data['projection_adp'].value_counts()
        if len(adp_counts) == 0:
            adp = 216.0
        else:
            adp = adp_counts.index[0]
        bbm4_adp.loc[idx, date] = adp

In [6]:
bbm4_adp.to_csv(r'C:\Users\Jona\Desktop\tuxff\data\bbm4_adp.csv')
bbm4_adp.head(5)

NameError: name 'bbm4_adp' is not defined

### ADP Data -> Postgres DB

In [5]:
bbm4_adp = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\adp_data\bbm4_adp.csv', delimiter=',')
bbm4_adp.head(5)

,Unnamed: 0,player_id,player_name,position,2023-04-29,2023-04-30,2023-05-01,2023-05-02,2023-05-03,2023-05-04,...,2023-08-28,2023-08-29,2023-08-30,2023-08-31,2023-09-01,2023-09-02,2023-09-03,2023-09-04,2023-09-05,2023-09-06
0,0,25ea41e7-0529-48c6-ba65-7648bb7f9ed2,Stefon Diggs,WR,8.0,8.64,8.25,8.00,7.91,7.99,...,8.87,8.84,8.74,8.77,8.72,8.61,8.57,8.56,8.29,7.83
1,1,99398db6-2ffb-42e9-bdb0-4312a399ab08,Josh Allen,QB,14.0,15.51,15.85,16.55,17.23,17.61,...,26.73,26.86,27.13,27.36,27.43,27.42,27.26,27.03,26.92,26.64
2,2,d74acd73-504a-4c9b-8aa6-fe602de560a8,Keenan Allen,WR,52.0,52.71,52.89,53.00,52.87,52.47,...,35.73,35.76,35.41,35.09,35.00,34.97,34.95,35.00,35.06,35.01
3,3,c108c752-92fd-491f-94ed-60b0b3cea7d8,Christian Watson,WR,48.0,48.17,47.88,47.17,46.13,45.85,...,40.35,40.26,40.03,39.70,39.60,39.70,39.76,39.72,39.39,39.18
4,4,660a9e72-4fc7-442d-ba62-257dbab39568,T.J. Hockenson,TE,37.0,39.81,40.84,41.93,42.99,43.84,...,54.02,53.96,53.75,53.82,53.32,52.98,53.16,52.88,52.04,51.45


In [28]:
for idx, row in bbm4_adp.iterrows():
    cursor.execute("""
        INSERT INTO player_info (player_id, player_name, position) 
        VALUES (%s, %s, %s)
        ON CONFLICT (player_id) DO NOTHING
    """, (row['player_id'], row['player_name'], row['position']))

conn.commit()

In [26]:
cursor.execute("SELECT player_id, date FROM player_adp")
existing_data = set()
for row in cursor.fetchall():
    existing_data.add((row['player_id'], row['date']))
print('Step 1 Complete')

adp_data = []
for idx, row in bbm4_adp.iterrows():
    for date in bbm4_adp.columns[4:]:
        if (row['player_id'], date) not in existing_data:
            adp_data.append((row['player_id'], date, row[date]))
print(f"Step 2 Complete - {len(adp_data)} new records to insert")

batch_size = 500
for i in range(0, len(adp_data), batch_size):
    batch = adp_data[i:i+batch_size]
    print(f"Starting batch {i//batch_size + 1} with {len(batch)} records...")
    cursor.executemany("""
        INSERT INTO player_adp (player_id, date, adp) 
        VALUES (%s, %s, %s)
    """, batch)
    print(f"Completed batch {i//batch_size + 1}")
    
    conn.commit()
    print(f"Committed batch {i//batch_size + 1}")

Step 1 Complete
Step 2 Complete - 96940 new records to insert
Starting batch 1 with 500 records...
Completed batch 1
Committed batch 1
Starting batch 2 with 500 records...
Completed batch 2
Committed batch 2
Starting batch 3 with 500 records...
Completed batch 3
Committed batch 3
Starting batch 4 with 500 records...
Completed batch 4
Committed batch 4
Starting batch 5 with 500 records...
Completed batch 5
Committed batch 5
Starting batch 6 with 500 records...
Completed batch 6
Committed batch 6
Starting batch 7 with 500 records...
Completed batch 7
Committed batch 7
Starting batch 8 with 500 records...
Completed batch 8
Committed batch 8
Starting batch 9 with 500 records...
Completed batch 9
Committed batch 9
Starting batch 10 with 500 records...
Completed batch 10
Committed batch 10
Starting batch 11 with 500 records...
Completed batch 11
Committed batch 11
Starting batch 12 with 500 records...
Completed batch 12
Committed batch 12
Starting batch 13 with 500 records...
Completed batch

In [27]:
conn.close()
cursor.close()

# BBM5 (2024)

In [22]:
bbm5_adp = pd.read_csv(r'C:\Users\Jona\Desktop\tuxff\data\adp_data\bbm5_adp.csv', delimiter=',')
bbm5_adp.head(5)

,Unnamed: 0,player_id,player_name,position,2024-04-29,2024-04-30,2024-05-01,2024-05-02,2024-05-03,2024-05-04,...,2024-08-22,2024-08-23,2024-08-24,2024-08-25,2024-08-26,2024-08-27,2024-08-28,2024-08-29,2024-08-30,2024-08-31
0,0,cb7b1dc6-91ce-47b8-813c-07486371a922,Christian McCaffrey,RB,1.0,1.11,1.15,1.18,1.20,1.22,...,1.23,1.22,1.21,1.21,1.23,1.27,1.30,1.30,1.29,1.27
1,1,4de8084d-aa1a-43c7-9676-0def5b39e980,CeeDee Lamb,WR,2.0,2.26,2.25,2.19,2.14,2.13,...,2.31,2.33,2.37,2.38,2.38,2.19,2.06,2.05,2.08,2.10
2,2,84cf37a2-d9f9-4315-a56a-c003a186023d,Tyreek Hill,WR,3.0,3.15,3.26,3.25,3.22,3.23,...,2.87,2.86,2.85,2.83,2.82,2.89,2.96,2.96,2.95,2.96
3,3,19818cc6-e21b-4258-9315-93cb64fb92e7,Ja'Marr Chase,WR,4.0,4.08,4.08,4.07,4.07,4.07,...,4.80,4.91,4.98,4.99,4.90,4.85,4.82,4.94,5.23,5.36
4,4,d61fb310-35c9-4f90-a99c-fb4605a870f4,Justin Jefferson,WR,6.0,5.82,5.65,5.47,5.32,5.26,...,6.27,6.28,6.27,6.26,6.32,6.50,6.55,6.44,6.23,6.27


In [18]:
for idx, row in bbm5_adp.iterrows():
    cursor.execute("""
        INSERT INTO player_info (player_id, player_name, position) 
        VALUES (%s, %s, %s)
        ON CONFLICT (player_id) DO UPDATE SET
            player_name = EXCLUDED.player_name,
            position = EXCLUDED.position
    """, (row['player_id'], row['player_name'], row['position']))

conn.commit()

In [ ]:
for idx, row in bbm5_adp.iterrows():
    for date in bbm5_adp.columns[4:]:
        cursor.execute("""
            INSERT INTO player_adp (player_id, date, adp) 
            VALUES (%s, %s, %s)
            ON CONFLICT (player_id) DO UPDATE SET
                player_name = EXCLUDED.player_name,
                position = EXCLUDED.position
        """, (row['player_id'], date, row[date]))

conn.commit()

# Add BBM6 players to postgres db

In [15]:
bbm6_data = pd.read_csv('data/Underdog_Draft_Table_2025-07-13.csv', delimiter=',')
bbm6_data.head()

,Rank,Player,Position,Position Rank,ADP on April 29,ADP on July 12,ADP Change
0,1,Ja'Marr Chase,WR,WR1,1.1,1.1,0.0
1,2,Justin Jefferson,WR,WR2,3.7,2.8,-0.9
2,3,Bijan Robinson,RB,RB1,3.7,3.1,-0.6
3,4,Saquon Barkley,RB,RB2,2.5,4.1,1.6
4,5,CeeDee Lamb,WR,WR3,5.3,4.8,-0.5


In [46]:
cursor.execute('SELECT player_name FROM player_info')
player_info = cursor.fetchall()

player_names = []
for player in player_info:
    player_names.append(player['player_name'])

player_data = []
for idx, row in bbm6_data.iterrows():
    if row['Player'] not in player_names:
        print(row['Player'])
        player_data.append((str(uuid.uuid4()), row['Player'], row['Position']))

cursor.executemany(
    "INSERT INTO player_info (player_id, player_name, position) VALUES (%s, %s, %s)",
    player_data
)

conn.commit()

In [47]:
conn.close()
cursor.close()